# 2.1 DoWhy：在 Lalonde 数据上估计 ATE（平均因果效应）

本 Notebook 演示用 **DoWhy** 在经典 **Lalonde** 数据集上完成：
1) 构建因果模型（DAG / common causes）
2) 识别并估计 **ATE**（以 PSM 为例）
3) 做简单的稳健性/敏感性检验

---

### 环境准备（如已安装可跳过）
```bash
pip install -U dowhy pydot matplotlib pandas scikit-learn
```
> 注：`model.view_model()` 需要 `graphviz`/`pydot` 支持。

In [ ]:
# ① 加载内置 Lalonde 数据集
import dowhy.datasets
import pandas as pd

data = dowhy.datasets.lalonde_binary()
df = data['df']
print(df.head())
print('\nColumns:', list(df.columns))


**变量说明（常见列）**

- `treatment`：是否接受就业培训（0/1）
- `re78`：1978 年收入（结果）
- 其他：`age, educ, black, hispan, married, nodegree, re74, re75` 作为基线协变量（混杂）

In [ ]:
# ② 构建因果模型（声明混杂）
from dowhy import CausalModel

model = CausalModel(
    data=df,
    treatment='treatment',
    outcome='re78',
    common_causes=['age','educ','black','hispan','married','nodegree','re74','re75']
)

identified_estimand = model.identify_effect()
print('=== Identified Estimand ===')
print(identified_estimand)

# 如需可视化 DAG，请取消下行注释（需安装 graphviz/pydot）
# model.view_model()

In [ ]:
# ③ 估计 ATE（示例：倾向评分匹配 PSM）
estimate_psm = model.estimate_effect(
    identified_estimand,
    method_name='backdoor.propensity_score_matching'
)
print('ATE (PSM):', estimate_psm.value)


In [ ]:
# ④ 稳健性/敏感性检验（示例：随机添加共同原因）
refute = model.refute_estimate(
    identified_estimand,
    estimate_psm,
    method_name='random_common_cause'
)
print(refute)

---
### 附：倾向评分分布可视化（可选）
便于检查处理/对照组的重叠性。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

# 简单用逻辑回归估计 PS
X = df[['age','educ','black','hispan','married','nodegree','re74','re75']]
T = df['treatment'].values
logit = LogisticRegression(max_iter=1000)
logit.fit(X, T)
ps = logit.predict_proba(X)[:,1]

plt.figure()
plt.hist(ps[T==1], bins=30, alpha=0.6, label='Treated', density=True)
plt.hist(ps[T==0], bins=30, alpha=0.6, label='Control', density=True)
plt.xlabel('Propensity Score')
plt.ylabel('Density')
plt.title('Propensity Score Distributions (Pre-Matching)')
plt.legend()
plt.show()
